# 프로젝트: 얼굴에 캐릭 씌우기

### face detection vs face recognition
### 얼굴 찾기 vs 누구 얼굴인지 알아내기

In [1]:
import cv2
import mediapipe as mp


In [2]:
# 얼굴을 찾고, 찾은 얼굴에 표시를 하기 위한 변수 정의
mp_face_detection = mp.solutions.face_detection # 얼굴 검출은 위한 face detection 모듈
mp_drawing = mp.solutions.drawing_utils # 얼굴의 특징을 그리기 위한 drawing utils 모듈


In [3]:

# For static images:
# IMAGE_FILES = []
# with mp_face_detection.FaceDetection(
#     model_selection=1, min_detection_confidence=0.5) as face_detection:
#   for idx, file in enumerate(IMAGE_FILES):
#     image = cv2.imread(file)
#     # Convert the BGR image to RGB and process it with MediaPipe Face Detection.
#     results = face_detection.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

#     # Draw face detections of each face.
#     if not results.detections:
#       continue
#     annotated_image = image.copy()
#     for detection in results.detections:
#       print('Nose tip:')
#       print(mp_face_detection.get_key_point(
#           detection, mp_face_detection.FaceKeyPoint.NOSE_TIP))
#       mp_drawing.draw_detection(annotated_image, detection)
#     cv2.imwrite('/tmp/annotated_image' + str(idx) + '.png', annotated_image)


In [8]:

# For webcam input:
# cap = cv2.VideoCapture(0)
cap = cv2.VideoCapture('face_video.mp4')

# model selection : 0 or 1
# 0: 카메라로부터 2m 이내, 1: 카메라로부터 5m 이내에 적합
# min_detection_confidence(≒threshold) : 몇퍼 이상이면 얼굴
with mp_face_detection.FaceDetection(
    model_selection=0, min_detection_confidence=0.95) as face_detection:
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            print("Ignoring empty camera frame.")
            # If loading a video, use 'break' instead of 'continue'.
            break

        # To improve performance, optionally mark the image as not writeable to
        # pass by reference.
        image.flags.writeable = False
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = face_detection.process(image)

        # Draw the face detection annotations on the image.
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        if results.detections:
            for detection in results.detections:
                # mp_drawing.draw_detection(image, detection)
                
                # 6점(keypoints): 우눈, 좌눈, 코, 입, 우귀, 좌귀
                keypoints = detection.location_data.relative_keypoints
                right_eye = keypoints[0]
                left_eye = keypoints[1]
                nose = keypoints[2]
                
                h,w,_ = image.shape
                right_eye = (int(right_eye.x * w), int(right_eye.y * h))
                left_eye = (int(left_eye.x * w), int(left_eye.y * h))
                nose = (int(nose.x * w), int(nose.y * h))
                
                cv2.circle(image, right_eye, 50, (255,0,0), 5, cv2.LINE_AA)
                cv2.circle(image, left_eye, 50, (0,255,0), 5, cv2.LINE_AA)
                cv2.circle(image, nose, 20, (0,255,255), 5, cv2.LINE_AA)
                
                
        # Flip the image horizontally for a selfie-view display.
        cv2.imshow('MediaPipe Face Detection', cv2.resize(image, None, fx=0.8, fy=0.8))
        if cv2.waitKey(1) == ord('q'):
            break
cap.release()
cv2.destroyAllWindows()